In [26]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf
import keras
from keras.layers import LSTM, Dropout, Dense

In [13]:
df = pd.DataFrame()
df['Timestamp'] = []
df['LeftX'] = []
df['LeftY']= []
df['LeftVelX']= []
df['LeftVelY']= []
df['RightX']= []
df['RightY']= []
df['RightVelX']= []
df['RightVelY']= []
df['LeftPupilRadius']= []
df['RightPupilRadius']= []
df['LeftRadialVel']= []
df['LeftRadialAcc']= []
df['LeftRadialJerk']= []
df['RightRadialVel']= []
df['RightRadialAcc']= []
df['RightRadialJerk']= []
df['VVE_NumMeasuresAbnormal'] = []
display(df)

,Timestamp,LeftX,LeftY,LeftVelX,LeftVelY,RightX,RightY,RightVelX,RightVelY,LeftPupilRadius,RightPupilRadius,LeftRadialVel,LeftRadialAcc,LeftRadialJerk,RightRadialVel,RightRadialAcc,RightRadialJerk,VVE_NumMeasuresAbnormal


In [14]:
from os import path

filePath = 'data-normalized/'
fileCount = 0
scanData = []
scanNP_list =[]

yList = []
vve = pd.read_csv("VVE-clinical-devset.csv")
for index, row in vve.iterrows():
    fileName = row['ScanFileName']
    fnLen = len(fileName)
    modFileName = fileName[0:fnLen-4] + "_norm.csv"
    scanFile = filePath + modFileName
    if path.exists(scanFile):
        numMeasuresAbnormal = []      
        yList.append(row['VVE_NumMeasuresAbnormal'])
        #dataframeY = dataframeY.append({'VVEScore':row['VVE_NumMeasureAbnormal']})
        fileCount += 1
        scanContents = pd.read_csv(scanFile)
        scanNP = scanContents.to_numpy()
        scanData.append(scanContents)  
        scanNP_list.append(scanNP)
        #dfx = pd.DataFrame(scanNP)
        #dfy = dfx.apply(lambda x: x.str.strip() if isinstance(x, str) else x).replace('', np.nan)

        for i in range(100001):
          numMeasuresAbnormal.append(yList[len(yList) - 1])
        scanContents['VVE_NumMeasuresAbnormal'] = numMeasuresAbnormal

        df = df.append(scanContents)

      
        #df = df.append({'patient#': modFileName, 'LeftX': dfy[1], 'LeftY': dfy[2], 'LeftVelX': dfy[3], 'LeftVelY': dfy[4], 'RightX': dfy[5], 
                        #'RightY': dfy[6], 'RightVelX': dfy[7], 'RightVelY': dfy[8], 'LeftPupilRadius': dfy[9], 
                        #'RightPupilRadius': dfy[10], "LeftRadialVel": dfy[11], 'LeftRadialAcc': dfy[12], 
                        #'LeftRadialJerk': dfy[13], 'RightRadialVel': dfy[14], 
                        #'RightRadialAcc': dfy[15], 'RightRadialJerk': dfy[16]}, ignore_index=True)
    else:
       print(scanFile)

        #display(dfy)
        #if scanContents['LeftX'].notnull().values.sum() < 1000:
          #print(scanFile + '\n')

display(df)
print(fileCount)

print(index)
print(yList)
#train, test = train_test_split(df, test_size=0.2)



#display(train)

#print(str(fileCount) + ' datafiles found.')
#print(scanData)
#scanArray = np.dstack(scanNP_list)
#np.save(filePath + 'scanArray', scanArray)


C:\Users\evanb\AppData\Local\Temp\ipykernel_14540\2521552802.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(scanContents)
C:\Users\evanb\AppData\Local\Temp\ipykernel_14540\2521552802.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(scanContents)
C:\Users\evanb\AppData\Local\Temp\ipykernel_14540\2521552802.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(scanContents)
C:\Users\evanb\AppData\Local\Temp\ipykernel_14540\2521552802.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(scanContents)
C:\Users\evanb\AppData\Local\Temp\ipykernel_14540\2521552802.py:31: Futu

,Timestamp,LeftX,LeftY,LeftVelX,LeftVelY,RightX,RightY,RightVelX,RightVelY,LeftPupilRadius,RightPupilRadius,LeftRadialVel,LeftRadialAcc,LeftRadialJerk,RightRadialVel,RightRadialAcc,RightRadialJerk,VVE_NumMeasuresAbnormal,Unnamed: 17
0,10364.0,-1.12012,0.822296,-0.476191,0.259478,-1.23556,0.81014,-0.006763,0.000328,0.922345,0.883619,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,10366.0,-1.11850,0.825035,0.467160,0.521251,-1.23481,0.81014,0.230378,0.000328,0.921844,0.883619,0.003186,NaN,NaN,0.000751,NaN,NaN,0.0,NaN
2,10368.0,-1.11525,0.829143,0.938259,0.782385,-1.23406,0.81014,0.230668,0.000328,0.921343,0.883080,0.005239,0.002052,NaN,0.000751,9.163670e-07,NaN,0.0,NaN
3,10370.0,-1.11199,0.834621,0.938547,1.044160,-1.23256,0.81014,0.467520,0.000328,0.921343,0.882540,0.006371,0.001132,-0.000920,0.001501,7.495880e-04,0.000749,0.0,NaN
4,10372.0,-1.10955,0.838729,0.702853,0.782704,-1.23106,0.81014,0.467520,0.000328,0.921844,0.881999,0.004779,-0.001592,-0.002725,0.001501,0.000000e+00,-0.000750,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,210020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
99997,210022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
99998,210024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
99999,210026.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN


219
218
[0, 6, 8, 1, 8, 5, 6, 1, 3, 6, 0, 4, 1, 4, 7, 8, 5, 0, 3, 9, 9, 3, 7, 0, 0, 1, 8, 2, 0, 8, 0, 4, 7, 9, 0, 0, 0, 0, 7, 0, 0, 3, 2, 9, 7, 7, 7, 7, 9, 6, 1, 0, 4, 3, 4, 8, 0, 4, 3, 3, 4, 0, 0, 9, 4, 6, 5, 4, 3, 0, 2, 3, 4, 6, 5, 0, 3, 5, 5, 4, 5, 1, 5, 6, 2, 0, 2, 0, 5, 5, 5, 6, 3, 4, 1, 6, 2, 2, 0, 0, 3, 0, 2, 1, 1, 1, 5, 3, 3, 3, 2, 3, 6, 3, 5, 1, 0, 3, 5, 4, 5, 4, 3, 1, 3, 1, 6, 2, 5, 6, 1, 1, 0, 5, 0, 1, 2, 1, 0, 0, 0, 2, 5, 3, 2, 3, 4, 4, 3, 4, 2, 2, 0, 0, 4, 2, 4, 5, 4, 1, 5, 1, 0, 0, 5, 5, 4, 0, 6, 2, 0, 0, 3, 5, 4, 5, 3, 5, 5, 5, 3, 4, 5, 1, 3, 5, 6, 5, 0, 2, 6, 0, 0, 1, 5, 1, 1, 5, 0, 0, 3, 0, 0, 6, 3, 5, 0, 6, 1, 0, 1, 0, 5, 5, 0, 3, 2, 3, 5]


In [15]:
df.drop(columns = 'Unnamed: 17', inplace = True)
df.reset_index(drop = True, inplace = True)

In [16]:
df2 = df.head(7500075)

In [17]:
df2

,Timestamp,LeftX,LeftY,LeftVelX,LeftVelY,RightX,RightY,RightVelX,RightVelY,LeftPupilRadius,RightPupilRadius,LeftRadialVel,LeftRadialAcc,LeftRadialJerk,RightRadialVel,RightRadialAcc,RightRadialJerk,VVE_NumMeasuresAbnormal
0,10364.0,-1.12012,0.822296,-0.476191,0.259478,-1.23556,0.810140,-0.006763,0.000328,0.922345,0.883619,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,10366.0,-1.11850,0.825035,0.467160,0.521251,-1.23481,0.810140,0.230378,0.000328,0.921844,0.883619,0.003186,NaN,NaN,0.000751,NaN,NaN,0.0
2,10368.0,-1.11525,0.829143,0.938259,0.782385,-1.23406,0.810140,0.230668,0.000328,0.921343,0.883080,0.005239,2.052210e-03,NaN,0.000751,9.163670e-07,NaN,0.0
3,10370.0,-1.11199,0.834621,0.938547,1.044160,-1.23256,0.810140,0.467520,0.000328,0.921343,0.882540,0.006371,1.132280e-03,-9.199280e-04,0.001501,7.495880e-04,0.000749,0.0
4,10372.0,-1.10955,0.838729,0.702853,0.782704,-1.23106,0.810140,0.467520,0.000328,0.921844,0.881999,0.004779,-1.592240e-03,-2.724520e-03,0.001501,0.000000e+00,-0.000750,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10000095,210020.0,-1.11395,1.078520,-0.000652,0.347911,-1.09507,0.967742,-0.187776,0.551339,0.764655,0.726082,0.001133,-3.117420e-04,-6.227910e-04,0.002397,2.397170e-03,0.003515,0.0
10000096,210022.0,-1.11395,1.079660,-0.000652,0.348123,-1.09593,0.968860,-0.187890,0.276075,0.764051,0.726082,0.001134,6.919800e-07,3.124340e-04,0.001413,-9.839820e-04,-0.003381,0.0
10000097,210024.0,-1.11395,1.080790,-0.000652,0.347911,-1.09507,0.971096,0.186837,0.551339,0.764655,0.726082,0.001133,-6.919800e-07,-1.383960e-06,0.002397,9.841720e-04,0.001968,0.0
10000098,210026.0,-1.11395,1.079660,-0.000652,-0.344468,-1.09593,0.971096,-0.187890,0.000979,0.764655,0.726738,0.001133,0.000000e+00,6.919800e-07,0.000865,-1.532470e-03,-0.002517,0.0


In [18]:
print(df2['VVE_NumMeasuresAbnormal'].unique())

[0. 6. 8. 1. 5. 3. 4. 7. 9. 2.]


In [19]:
rec_count = len(df2)
rec_count

10000100

In [20]:
sequence_length = 10

In [23]:
def generate_data(X, y, sequence_length = 10, step = 1):
    X_local = []
    y_local = []
    for start in range(0, len(df2) - sequence_length, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

X_sequence, y = generate_data(df2.loc[:, "LeftX":"RightRadialJerk"].values, df2.VVE_NumMeasuresAbnormal)

In [24]:
X_sequence.shape, y.shape

((10000090, 10, 16), (10000090,))

In [31]:
model = keras.Sequential()
model.add(LSTM(100, input_shape = (10, 16)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               46800     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 46,901
Trainable params: 46,901
Non-trainable params: 0
_________________________________________________________________


In [32]:
training_size = int(len(X_sequence) * 0.7)

In [33]:
X_train, y_train = X_sequence[:training_size], y[:training_size]
X_test, y_test = X_sequence[training_size:], y[training_size:]

In [34]:
model.fit(X_train, y_train, batch_size=64, epochs=50)

Epoch 1/50
109376/109376 [==============================] - 1294s 12ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 2/50
109376/109376 [==============================] - 1199s 11ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 3/50
109376/109376 [==============================] - 1405s 13ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 4/50
109376/109376 [==============================] - 1296s 12ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 5/50
109376/109376 [==============================] - 1239s 11ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 6/50
109376/109376 [==============================] - 1731s 16ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 7/50
109376/109376 [==============================] - 1766s 16ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 8/50
109376/109376 [==============================] - 1749s 16ms/step - loss: nan - binary_accuracy: 0.2571
Epoch 9/50
109376/109376 [==============================] - 1735s 16ms/step - loss: nan 

In [35]:
model.evaluate(X_test, y_test)

93751/93751 [==============================] - 336s 4ms/step - loss: nan - binary_accuracy: 0.1667


[nan, 0.16666649281978607]

In [36]:
y_test_prob = model.predict(X_test, verbose=1)

93751/93751 [==============================] - 291s 3ms/step


In [37]:
y_test_pred = np.where(y_test_prob > 0.5, 1, 0)

In [39]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_test_pred)

array([[500004,      0,      0,      0,      0,      0,      0],
       [200002,      0,      0,      0,      0,      0,      0],
       [500003,      0,      0,      0,      0,      0,      0],
       [300003,      0,      0,      0,      0,      0,      0],
       [300003,      0,      0,      0,      0,      0,      0],
       [800008,      0,      0,      0,      0,      0,      0],
       [400004,      0,      0,      0,      0,      0,      0]],
      dtype=int64)

In [41]:
y_test_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])